__Инструкция__

Загрузите данные и посчитайте модели линейной регрессии для 50 зданий по ансамблю регрессионных моделей: в первой модели весь оптимальный набор метеорологических данных, во второй - дни недели и праздники, в третьей - недели года, в четвертой - месяцы. Финальное значение показателя рассчитайте как взвешенное арифметическое среднее показателей всех моделей, взяв веса для первой и второй модели как 3/8, а для третьей и четвертой - как 1/8.

Загрузите данные решения, посчитайте значение энергопотребления для требуемых дат для тех зданий, которые посчитаны в модели, и выгрузите результат в виде CSV-файла (submission.csv).

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/test.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_test.csv.gz

Итоговый файл с кодом (.py или .ipynb) выложите в github с портфолио.

__Постановка задачи__
1. Загрузите данные (building_metadata.csv.gz, weather_train.csv.gz, train.0.csv.gz).
2. Посчитайте модели линейной регрессии для 50 зданий по ансамблю регрессионных моделей: 
* в первой модели весь оптимальный набор метеорологических данных
* во второй - дни недели и праздники
* в третьей - недели года
* в четвертой - месяцы
3. Рассчитайте финальное значение показателя как взвешенное арифметическое среднее показателей всех моделей, взяв веса:
 * для первой и второй модели как 3/8
 * для третьей и четвертой - как 1/8
4. Загрузите данные решения (test.csv.gz, weather_test.csv.gz).
5. Посчитайте значение энергопотребления для требуемых дат для тех зданий, которые посчитаны в модели.
6. Выгрузите результат в виде CSV-файла (submission.csv).

__Подключим все необходимые библиотеки и добавим функцию оптимизации памяти__

In [56]:
# библиотеки
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import numpy as np
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import os

# Функция  оптимизации памяти
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

#### 1. Загрузите данные (building_metadata.csv.gz, weather_train.csv.gz, train.0.csv.gz)
Загрузим данные, проведем ETL

In [57]:
# загрузка данных из локального источника и выбор данных по погоде, только для города 0, данных по зданиям только первых 50 зданий
buildings = pd.read_csv("building_metadata.csv.gz")
weather = pd.read_csv("weather_train.csv.gz")
weather = weather[weather["site_id"] == 0]  # только 1 город
energy = pd.read_csv("train.0.csv.gz")
energy = energy[energy["building_id"] < 50]  #  только первые 50 зданий
energy = energy[energy["meter_reading"] > 0] 
# объединение данных потребления и по зданиям
energy = pd.merge(left=energy, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
del buildings  # удаление промежуточных данных

In [58]:
# обогащение данных погода, проведем интерполяцию серий и добавим первую и вторую производную температуры воздуха

# интерполяция значений данных погода, только оптимальных серий
interpolate_columns = ["air_temperature", "dew_temperature", "cloud_coverage", 
                       "wind_speed", "wind_direction", "sea_level_pressure"]
for col in interpolate_columns:
    weather[col] = weather[col].interpolate(limit_direction='both',
                            kind='cubic') 
# обогащение данных погода, производными от температуры
weather["air_temperature_diff1"] = weather["air_temperature"].diff()
weather.at[0, "air_temperature_diff1"] = weather.at[1, "air_temperature_diff1"]
weather["air_temperature_diff2"] = weather["air_temperature_diff1"].diff()
weather.at[0, "air_temperature_diff2"] = weather.at[1, "air_temperature_diff2"]
weather = weather.drop(columns=["precip_depth_1_hr", "wind_direction"], axis=1) # удалим серии которые нам не понадобятся
print(weather.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8784 entries, 0 to 8783
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   site_id                8784 non-null   int64  
 1   timestamp              8784 non-null   object 
 2   air_temperature        8784 non-null   float64
 3   cloud_coverage         8784 non-null   float64
 4   dew_temperature        8784 non-null   float64
 5   sea_level_pressure     8784 non-null   float64
 6   wind_speed             8784 non-null   float64
 7   air_temperature_diff1  8784 non-null   float64
 8   air_temperature_diff2  8784 non-null   float64
dtypes: float64(7), int64(1), object(1)
memory usage: 944.3+ KB
None


In [59]:
# Объединение данных потребления и погоды через индексы
energy.set_index(["site_id", "timestamp"], inplace=True )
weather.set_index(["site_id", "timestamp"], inplace=True )
energy = pd.merge(left=energy, right=weather, how="left",
                    left_index=True, right_index=True)
energy.reset_index(inplace=True)
# удалим не нужные колонки
energy = energy.drop(columns=["meter", "year_built",
                              "square_feet", "floor_count"], axis=1)

# оптимизация потребления памяти и удаление промежуточных данных 
energy = reduce_mem_usage(energy)
del weather
print (energy.info())

Потребление памяти меньше на 17.71 Мб (минус 71.9 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269067 entries, 0 to 269066
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   site_id                269067 non-null  int8          
 1   timestamp              269067 non-null  datetime64[ns]
 2   building_id            269067 non-null  int8          
 3   meter_reading          269067 non-null  float16       
 4   primary_use            269067 non-null  category      
 5   air_temperature        269067 non-null  float16       
 6   cloud_coverage         269067 non-null  float16       
 7   dew_temperature        269067 non-null  float16       
 8   sea_level_pressure     269067 non-null  float16       
 9   wind_speed             269067 non-null  float16       
 10  air_temperature_diff1  269067 non-null  float16       
 11  air_temperature_diff2  269067 non-null  float16    

In [60]:
# добавление серий данных час, день недели, неделя, месяц, дата и праздничные дни. 
energy["hour"] = energy["timestamp"].dt.hour.astype("int8")
energy["weekday"] = energy["timestamp"].dt.weekday.astype("int8")
energy["week"] = energy["timestamp"].dt.isocalendar().week.astype("int8")
energy["month"] = energy["timestamp"].dt.month.astype("int8")
energy["date"] = pd.to_datetime(energy["timestamp"].dt.date)
dates_range = pd.date_range(start='2015-12-31', end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(),
                                  end=dates_range.max())
energy['is_holiday'] = energy['date'].isin(us_holidays).astype("int8")
for weekday in range(0,7):
    energy['is_wday' + str(weekday)] = energy['weekday'].isin([weekday]).astype("int8")
for week in range(1,54):
    energy['is_w' + str(week)] = energy['week'].isin([week]).astype("int8")
for month in range(1,13):
    energy['is_m' + str(month)] = energy['month'].isin([month]).astype("int8")
    
# добавили серию с логарифмом целевого показателя 
energy["meter_reading_log"] = np.log(energy["meter_reading"] + 1)

#### 2. Посчитайте модели линейной регрессии для 50 зданий по ансамблю регрессионных моделей:

* в первой модели весь оптимальный набор метеорологических данных
* во второй - дни недели и праздники
* в третьей - недели года
* в четвертой - месяцы

In [61]:

hours = range(0, 24)  # диапазон для итераций по часам
buildings = range(0, energy["building_id"].max() + 1) # диапазон для итераций кол-ву зданий
def train_model (df, columns):  # функция для расчета коэффициентов модели линейной регрессии (построения модели)
    df_train_lr = pd.DataFrame(df, columns=columns)
    df_lr = [[]] * len(buildings)
    for building in buildings:
        df_lr[building] =  [[]] * len(hours)
        df_train_b = df_train_lr[df_train_lr["building_id"] == building]
        for hour in hours:
            df_train_bh = df_train_b[df_train_b["hour"] == hour]
            y = df_train_bh["meter_reading_log"]
            x = df_train_bh.drop(labels=["meter_reading_log","hour", "building_id"], axis=1)
            model = LinearRegression(fit_intercept=False).fit(x, y)
            df_lr[building][hour] = model.coef_
            df_lr[building][hour] = np.append(df_lr[building][hour], model.intercept_)
    return df_lr

In [62]:
# Модель линейной регрессии по набору метеорологических данных (по погоде)
lr_weather_columns = ["meter_reading_log","hour", "building_id", "air_temperature", "cloud_coverage", "dew_temperature",
            "sea_level_pressure", "wind_speed", "air_temperature_diff1", "air_temperature_diff2"]
lr_weather = train_model(energy, lr_weather_columns)
# print(lr_weather[0])


In [63]:
# Модель линейной регрессии по набору дни недели и праздники
lr_wday_holiday_columns = ["meter_reading_log","hour", "building_id", "is_holiday"]
for wday in range(0, 7):
    lr_wday_holiday_columns.append("is_wday" + str(wday))
lr_wday_holiday = train_model(energy, lr_wday_holiday_columns)
# print(lr_wday_holiday[0])

In [64]:
# Модель линейной регрессии по набору недели года
lr_week_columns = ["meter_reading_log","hour", "building_id"]
for week in range(1, 54):
    lr_week_columns.append("is_w" + str(week))
lr_week = train_model(energy, lr_week_columns)
# print(lr_week[0])

In [65]:
# Модель линейной регрессии по набору месяцы
lr_month_columns = ["meter_reading_log","hour", "building_id"]
for month in range(1, 13):
    lr_month_columns.append("is_m" + str(month))
lr_month = train_model(energy, lr_month_columns)
# print(lr_month[0])

In [66]:
del energy  # удаление промежуточных данных

#### 3. Финальное значение показателя рассчитайте как взвешенное арифметическое среднее показателей всех моделей, взяв веса:
 * для первой и второй модели как 3/8
 * для третьей и четвертой - как 1/8

In [74]:
# добавим функции для расчета финального значения 
def calculate_model_ensemble (x, model, columns):
    lr = -1
    if len(model) > 0:
        lr = np.sum([x[col] * model[i] for i,col in enumerate(columns[3:])])
        lr += model[len(columns)-3]
        lr = np.exp(lr)
    if lr < 0 or lr*lr == lr:
        lr = 0
    return lr

def calculate_models_ensemble_V1 (x):  # функция для расчета финального значения показателя
    lr_weath = calculate_model_ensemble(x,
            lr_weather[x.building_id][x.hour],
            lr_weather_columns)
    lr_wday_hol = calculate_model_ensemble(x,
            lr_wday_holiday[x.building_id][x.hour],
            lr_wday_holiday_columns)
    lr_w = calculate_model_ensemble(x,
            lr_week[x.building_id][x.hour],
            lr_week_columns)
    lr_m = calculate_model_ensemble(x,
            lr_month[x.building_id][x.hour],
            lr_month_columns)
    lr_mean = (lr_weath*3 + lr_wday_hol*3 + lr_w + lr_m)/8
    x["meter_reading"] = lr_mean
    return x

#### 4. Загрузите данные решения
Загрузим данные решения (test.csv.gz, weather_test.csv.gz), проведем ETL.


In [75]:
buildings = pd.read_csv("building_metadata.csv.gz",
                       usecols=["site_id", "building_id"])
weather = pd.read_csv("weather_test.csv.gz")
weather = weather[weather["site_id"] == 0]
results = pd.read_csv("test.csv.gz")
results = results[(results["building_id"] < 50) & (results["meter"] == 0)]
results = pd.merge(left=results, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
del buildings
results = results.drop(columns=["meter"], axis=1)
# print (results.info())

In [76]:
# отберем колонки для интерполяции данных погоды
interpolate_columns = ["air_temperature", "dew_temperature",
                       "cloud_coverage", "wind_speed", "sea_level_pressure"]
# проведем интерполяцию 
for col in interpolate_columns:
    weather[col] = weather[col].interpolate(limit_direction='both',
                            kind='cubic') 
# обогащение данных погода, производными от температуры
weather["air_temperature_diff1"] = weather["air_temperature"].diff()
weather.at[0, "air_temperature_diff1"] = weather.at[1, "air_temperature_diff1"]
weather["air_temperature_diff2"] = weather["air_temperature_diff1"].diff()
weather.at[0, "air_temperature_diff2"] = weather.at[1, "air_temperature_diff2"]
weather = weather.drop(columns=["precip_depth_1_hr", "wind_direction"], axis=1) # удалим серии которые нам не понадобятся
print(weather.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17520 entries, 0 to 17519
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   site_id                17520 non-null  int64  
 1   timestamp              17520 non-null  object 
 2   air_temperature        17520 non-null  float64
 3   cloud_coverage         17520 non-null  float64
 4   dew_temperature        17520 non-null  float64
 5   sea_level_pressure     17520 non-null  float64
 6   wind_speed             17520 non-null  float64
 7   air_temperature_diff1  17520 non-null  float64
 8   air_temperature_diff2  17520 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 1.8+ MB
None


In [77]:
# объединим данные по погоде
results = results.set_index(["timestamp", "site_id"])  # назначили индексы
weather = weather.set_index(["timestamp", "site_id"])
# объединили по индексам
results = pd.merge(left=results, right=weather, how="left",
                  left_index=True, right_index=True)
results.reset_index(inplace=True)  # сбросили индексы после объединения 
results = results.drop(columns=["site_id"], axis=1)  # удалили неиспользуемую серию данных 
del weather  # удалили промежуточные данные
results = reduce_mem_usage(results)  # провели оптимизацию памяти
print (results.info())

Потребление памяти меньше на 44.28 Мб (минус 66.2 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876000 entries, 0 to 875999
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              876000 non-null  datetime64[ns]
 1   row_id                 876000 non-null  int32         
 2   building_id            876000 non-null  int8          
 3   air_temperature        876000 non-null  float16       
 4   cloud_coverage         876000 non-null  float16       
 5   dew_temperature        876000 non-null  float16       
 6   sea_level_pressure     876000 non-null  float16       
 7   wind_speed             876000 non-null  float16       
 8   air_temperature_diff1  876000 non-null  float16       
 9   air_temperature_diff2  876000 non-null  float16       
dtypes: datetime64[ns](1), float16(7), int32(1), int8(1)
memory usage: 22.6 MB
None


In [78]:
# добавим необходимые серии час, день недели, неделя, месяц, день и праздник
results["hour"] = results["timestamp"].dt.hour.astype("int8")
results["weekday"] = results["timestamp"].dt.weekday.astype("int8")
results["week"] = results["timestamp"].dt.isocalendar().week.astype("int8")
results["month"] = results["timestamp"].dt.month.astype("int8")
results["date"] = pd.to_datetime(results["timestamp"].dt.date)
dates_range = pd.date_range(start='2016-12-31', end='2018-06-01')
us_holidays = calendar().holidays(start=dates_range.min(),
                                  end=dates_range.max())
results['is_holiday'] = results['date'].isin(us_holidays).astype("int8")
for weekday in range(0,7):
    results['is_wday' + str(weekday)] = results['weekday'].isin([weekday]).astype("int8")
for week in range(1,54):
    results['is_w' + str(week)] = results['week'].isin([week]).astype("int8")
for month in range(1,13):
    results['is_m' + str(month)] = results['month'].isin([month]).astype("int8")

#### 5. Посчитайте значение энергопотребления для требуемых дат для тех зданий, которые посчитаны в модели.
Расчет финальных показателей, только энергопотребление, только 50 первых зданий.

In [80]:
results = results.apply(calculate_models_ensemble_V1, axis=1, result_type="expand")

In [81]:
# Усечение данных до требуемого формата: row_id, meter_reading
results_ready = pd.DataFrame(results, columns=["row_id", "meter_reading"])
print (results_ready.info())
print (len(results_ready[results_ready["meter_reading"]>0]))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876000 entries, 0 to 875999
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   row_id         876000 non-null  int64  
 1   meter_reading  876000 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 13.4 MB
None
876000


In [82]:
results = pd.read_csv("test.csv.gz", usecols=["row_id"])  # загрузка данных решения, используем только колонку row_id
# объединим рассчитанные данные  
results = pd.merge(left=results, right=results_ready, how="left",
                  left_on="row_id", right_on="row_id")
results.fillna(value=0, inplace=True)
print (results.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 2 columns):
 #   Column         Dtype  
---  ------         -----  
 0   row_id         int64  
 1   meter_reading  float64
dtypes: float64(1), int64(1)
memory usage: 954.4 MB
None


### Выгрузка результатов в CSV файл
Итоговый файл занимает около 1 Гб

In [ ]:
# Выгрузка результатов в CSV файл
results.to_csv("submission.csv",index=False)

In [83]:
# освобождение памяти 
del results
del results_ready